# EE514 assignment part 1 starter code

Using sensors to predict activity. This part of the assignment uses the [ExtraSensory dataset](http://extrasensory.ucsd.edu/). You can download the dataset from [here](http://extrasensory.ucsd.edu/data/primary_data_files/ExtraSensory.per_uuid_features_labels.zip). The starter code expects that this dataset has been unpacked in a folder called `data` that is in the same parent folder as this notebook. You can read more about the dataset in [this README file](http://extrasensory.ucsd.edu/data/primary_data_files/README.txt).

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:

df =pd.read_csv("/content/drive/MyDrive/Extrasensory/data/1155FF54-63D3-4AB2-9863-8385D0BD0A13.features_labels.csv")#Just to check 
df.head()

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,raw_acc:magnitude_spectrum:log_energy_band1,raw_acc:magnitude_spectrum:log_energy_band2,raw_acc:magnitude_spectrum:log_energy_band3,raw_acc:magnitude_spectrum:log_energy_band4,raw_acc:magnitude_spectrum:spectral_entropy,raw_acc:magnitude_autocorrelation:period,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,proc_gyro:magnitude_stats:mean,proc_gyro:magnitude_stats:std,proc_gyro:magnitude_stats:moment3,proc_gyro:magnitude_stats:moment4,proc_gyro:magnitude_stats:percentile25,proc_gyro:magnitude_stats:percentile50,proc_gyro:magnitude_stats:percentile75,proc_gyro:magnitude_stats:value_entropy,proc_gyro:magnitude_stats:time_entropy,proc_gyro:magnitude_spectrum:log_energy_band0,proc_gyro:magnitude_spectrum:log_energy_band1,proc_gyro:magnitude_spectrum:log_energy_band2,proc_gyro:magnitude_spectrum:log_energy_band3,...,label:IN_A_CAR,label:ON_A_BUS,label:DRIVE_-_I_M_THE_DRIVER,label:DRIVE_-_I_M_A_PASSENGER,label:LOC_home,label:FIX_restaurant,label:PHONE_IN_POCKET,label:OR_exercise,label:COOKING,label:SHOPPING,label:STROLLING,label:DRINKING__ALCOHOL_,label:BATHING_-_SHOWER,label:CLEANING,label:DOING_LAUNDRY,label:WASHING_DISHES,label:WATCHING_TV,label:SURFING_THE_INTERNET,label:AT_A_PARTY,label:AT_A_BAR,label:LOC_beach,label:SINGING,label:TALKING,label:COMPUTER_WORK,label:EATING,label:TOILET,label:GROOMING,label:DRESSING,label:AT_THE_GYM,label:STAIRS_-_GOING_UP,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
0,1464129912,1.011438,0.012573,0.023013,0.041240,1.006184,1.010868,1.016028,0.935099,6.684536,5.045404,0.002765,0.009658,0.012629,0.099805,0.432150,4.100449,0.162703,-0.029586,0.002907,1.010972,0.005533,0.005385,0.012524,0.088952,-0.191380,-0.160822,0.014675,0.019652,0.044390,0.069752,0.011122,0.011958,0.013003,0.240147,6.411181,4.906840,3.596673,4.678782,4.286671,...,0.0,0.0,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,1.0,0.0,2
1,1464129950,1.011233,0.009356,-0.005622,0.016687,1.006338,1.010926,1.016657,1.732968,6.684569,5.043640,0.000767,0.003770,0.003330,0.044046,0.430518,0.139308,0.098149,-0.029709,0.002341,1.010757,0.006041,0.006082,0.009382,-0.113686,-0.014063,0.117530,0.013265,0.011520,0.026085,0.040920,0.010916,0.012099,0.013177,0.302317,6.539878,5.121756,4.182344,5.022812,3.801169,...,0.0,0.0,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,1.0,0.0,2
2,1464130031,1.013422,0.018068,-0.008593,0.039286,1.004077,1.012983,1.021926,1.464639,6.684453,5.044152,0.000671,0.009051,0.020124,0.081974,0.431532,5.612973,0.107736,0.056145,0.198593,0.991466,0.026232,0.026095,0.019610,0.136089,-0.023222,-0.350544,0.058436,0.053728,0.078939,0.114179,0.026110,0.043186,0.069787,1.586952,6.374701,5.263151,3.729888,4.644520,3.357239,...,0.0,0.0,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,1.0,0.0,2
3,1464130109,1.014891,0.016400,0.021383,0.038825,1.005934,1.014670,1.023000,1.440043,6.684483,5.043807,0.000653,0.018569,0.011195,0.059983,0.431113,3.109283,0.170375,0.014083,0.180070,0.997869,0.023530,0.031472,0.019009,0.386376,-0.104977,-0.491163,0.078685,0.154462,0.321156,0.497587,0.024645,0.041906,0.078418,0.595077,6.002274,5.51779

## Location of the .csv.gz files

In [ ]:
data_dir = Path('data')

## Some utility functions

The first one loads a pandas dataframe given a user UUID. The second one extracts specified feature columns $X$ and target column $y$ from a dataframe and converts these to numpy.

In [ ]:
def load_data_for_user(uuid):
    return pd.read_csv(data_dir / (uuid))

def get_features_and_target(df, feature_names, target_name):
    
    # select out features and target columns and convert to numpy
    X = df[feature_names].to_numpy()
    y = df[target_name].to_numpy()
    
    # remove examples with no label
    has_label = ~np.isnan(y)
    X = X[has_label,:]
    y = y[has_label]
    return X, y

## Load in some data 
Load in the data for a user and display the first few rows of the dataframe

In [ ]:
df = load_data_for_user('/content/drive/MyDrive/Extrasensory/data/1155FF54-63D3-4AB2-9863-8385D0BD0A13.features_labels.csv')
df.head()

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,raw_acc:magnitude_spectrum:log_energy_band1,raw_acc:magnitude_spectrum:log_energy_band2,raw_acc:magnitude_spectrum:log_energy_band3,raw_acc:magnitude_spectrum:log_energy_band4,raw_acc:magnitude_spectrum:spectral_entropy,raw_acc:magnitude_autocorrelation:period,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,proc_gyro:magnitude_stats:mean,proc_gyro:magnitude_stats:std,proc_gyro:magnitude_stats:moment3,proc_gyro:magnitude_stats:moment4,proc_gyro:magnitude_stats:percentile25,proc_gyro:magnitude_stats:percentile50,proc_gyro:magnitude_stats:percentile75,proc_gyro:magnitude_stats:value_entropy,proc_gyro:magnitude_stats:time_entropy,proc_gyro:magnitude_spectrum:log_energy_band0,proc_gyro:magnitude_spectrum:log_energy_band1,proc_gyro:magnitude_spectrum:log_energy_band2,proc_gyro:magnitude_spectrum:log_energy_band3,...,label:IN_A_CAR,label:ON_A_BUS,label:DRIVE_-_I_M_THE_DRIVER,label:DRIVE_-_I_M_A_PASSENGER,label:LOC_home,label:FIX_restaurant,label:PHONE_IN_POCKET,label:OR_exercise,label:COOKING,label:SHOPPING,label:STROLLING,label:DRINKING__ALCOHOL_,label:BATHING_-_SHOWER,label:CLEANING,label:DOING_LAUNDRY,label:WASHING_DISHES,label:WATCHING_TV,label:SURFING_THE_INTERNET,label:AT_A_PARTY,label:AT_A_BAR,label:LOC_beach,label:SINGING,label:TALKING,label:COMPUTER_WORK,label:EATING,label:TOILET,label:GROOMING,label:DRESSING,label:AT_THE_GYM,label:STAIRS_-_GOING_UP,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
0,1464129912,1.011438,0.012573,0.023013,0.041240,1.006184,1.010868,1.016028,0.935099,6.684536,5.045404,0.002765,0.009658,0.012629,0.099805,0.432150,4.100449,0.162703,-0.029586,0.002907,1.010972,0.005533,0.005385,0.012524,0.088952,-0.191380,-0.160822,0.014675,0.019652,0.044390,0.069752,0.011122,0.011958,0.013003,0.240147,6.411181,4.906840,3.596673,4.678782,4.286671,...,0.0,0.0,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,1.0,0.0,2
1,1464129950,1.011233,0.009356,-0.005622,0.016687,1.006338,1.010926,1.016657,1.732968,6.684569,5.043640,0.000767,0.003770,0.003330,0.044046,0.430518,0.139308,0.098149,-0.029709,0.002341,1.010757,0.006041,0.006082,0.009382,-0.113686,-0.014063,0.117530,0.013265,0.011520,0.026085,0.040920,0.010916,0.012099,0.013177,0.302317,6.539878,5.121756,4.182344,5.022812,3.801169,...,0.0,0.0,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,1.0,0.0,2
2,1464130031,1.013422,0.018068,-0.008593,0.039286,1.004077,1.012983,1.021926,1.464639,6.684453,5.044152,0.000671,0.009051,0.020124,0.081974,0.431532,5.612973,0.107736,0.056145,0.198593,0.991466,0.026232,0.026095,0.019610,0.136089,-0.023222,-0.350544,0.058436,0.053728,0.078939,0.114179,0.026110,0.043186,0.069787,1.586952,6.374701,5.263151,3.729888,4.644520,3.357239,...,0.0,0.0,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,1.0,0.0,2
3,1464130109,1.014891,0.016400,0.021383,0.038825,1.005934,1.014670,1.023000,1.440043,6.684483,5.043807,0.000653,0.018569,0.011195,0.059983,0.431113,3.109283,0.170375,0.014083,0.180070,0.997869,0.023530,0.031472,0.019009,0.386376,-0.104977,-0.491163,0.078685,0.154462,0.321156,0.497587,0.024645,0.041906,0.078418,0.595077,6.002274,5.51779

## What columns are available?

In [ ]:
print(df.columns.to_list())

['timestamp', 'raw_acc:magnitude_stats:mean', 'raw_acc:magnitude_stats:std', 'raw_acc:magnitude_stats:moment3', 'raw_acc:magnitude_stats:moment4', 'raw_acc:magnitude_stats:percentile25', 'raw_acc:magnitude_stats:percentile50', 'raw_acc:magnitude_stats:percentile75', 'raw_acc:magnitude_stats:value_entropy', 'raw_acc:magnitude_stats:time_entropy', 'raw_acc:magnitude_spectrum:log_energy_band0', 'raw_acc:magnitude_spectrum:log_energy_band1', 'raw_acc:magnitude_spectrum:log_energy_band2', 'raw_acc:magnitude_spectrum:log_energy_band3', 'raw_acc:magnitude_spectrum:log_energy_band4', 'raw_acc:magnitude_spectrum:spectral_entropy', 'raw_acc:magnitude_autocorrelation:period', 'raw_acc:magnitude_autocorrelation:normalized_ac', 'raw_acc:3d:mean_x', 'raw_acc:3d:mean_y', 'raw_acc:3d:mean_z', 'raw_acc:3d:std_x', 'raw_acc:3d:std_y', 'raw_acc:3d:std_z', 'raw_acc:3d:ro_xy', 'raw_acc:3d:ro_xz', 'raw_acc:3d:ro_yz', 'proc_gyro:magnitude_stats:mean', 'proc_gyro:magnitude_stats:std', 'proc_gyro:magnitude_stat

## Feature selection

The columns that start with `label:` correspond to potential y values. Let's look at using the accelerometer features. These start with `raw_acc:` and `watch_acceleration:`

In [ ]:
acc_sensors = [s for s in df.columns if 
               s.startswith('raw_acc:') or 
               s.startswith('watch_acceleration:')]

target_column = 'label:FIX_walking'

## Extract our training data

In [ ]:
X_train, y_train = get_features_and_target(df, acc_sensors, target_column)
print(f'{y_train.shape[0]} examples with {y_train.sum()} positives')

2681 examples with 158.0 positives


## Preprocessing

We want to make the learning problem easier by making all columns have a mean of zero and a standard deviation of one. There are also lots of missing values in this dataset. We'll use mean imputation here to get rid of them. Since our data is scaled to have zero mean, this will just zero out missing values.

In [ ]:
scaler = StandardScaler()
imputer = SimpleImputer(strategy='mean')

X_train = scaler.fit_transform(X_train)
X_train = imputer.fit_transform(X_train)

## Fitting a model
Let's fit a logistic regression model to this user. We can then test it's predictive power on a different user

In [ ]:
clf = LogisticRegression(solver='liblinear', max_iter=1000, C=1.0)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

## Training accuracy

Let's see the accuracy on the training set. The score function can be used to do this:

In [ ]:
print(f'Training accuracy: {clf.score(X_train, y_train):0.4f}')

Training accuracy: 0.9750


Looks like the model can fit the training data reasonably well anyway. But this says nothing about how well it will generalize to new data. The dataset is also unbalanced, so this figure may be misleading. How accurate would we be if we just predicted zero each time?

In [ ]:
1 - y_train.sum() / y_train.shape[0]

0.9410667661320403

Oh wow. Our model may not be that great after all. Let's try to calculate balanced accuracy, which should better reflect how well the model does on the training data

In [ ]:
y_pred = clf.predict(X_train)
print(f'Balanced accuracy (train): {metrics.balanced_accuracy_score(y_train, y_pred):0.4f}')

Balanced accuracy (train): 0.8354


## Testing the model

Ok, it seems our model has fit the training data well. How well does it perform on unseen test data? Let's load the data in for a different user.

In [ ]:
df_test = load_data_for_user('/content/drive/MyDrive/Extrasensory/data/11B5EC4D-4133-4289-B475-4E737182A406.features_labels.csv')
X_test, y_test = get_features_and_target(df_test, acc_sensors, target_column)
print(f'{y_train.shape[0]} examples with {y_train.sum()} positives')
X_test = imputer.transform(scaler.transform(X_test))
print(f'Test accuracy: {clf.score(X_test, y_test):0.4f}')




2681 examples with 158.0 positives
Test accuracy: 0.9746


Loading Data for 5 different users
Finding Mean and Vaiance of each

In [ ]:
df_test1 = load_data_for_user('/content/drive/MyDrive/Extrasensory/data/D7D20E2E-FC78-405D-B346-DBD3FD8FC92B.features_labels.csv')
X_test, y_test = get_features_and_target(df_test1, acc_sensors, target_column)
print(f'{y_train.shape[0]} examples with {y_train.sum()} positives')
X_test = imputer.transform(scaler.transform(X_test))
print(f'Test accuracy: {clf.score(X_test, y_test):0.4f}')
y_pred = clf.predict(X_test)
print(f'Balanced accuracy (train): {metrics.balanced_accuracy_score(y_test, y_pred):0.4f}')
pd.DataFrame([df_test1.mean(), df_test1.var()], index=['Mean', 'Variance'])

2681 examples with 158.0 positives
Test accuracy: 0.9615
Balanced accuracy (train): 0.7858


,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,raw_acc:magnitude_spectrum:log_energy_band1,raw_acc:magnitude_spectrum:log_energy_band2,raw_acc:magnitude_spectrum:log_energy_band3,raw_acc:magnitude_spectrum:log_energy_band4,raw_acc:magnitude_spectrum:spectral_entropy,raw_acc:magnitude_autocorrelation:period,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,proc_gyro:magnitude_stats:mean,proc_gyro:magnitude_stats:std,proc_gyro:magnitude_stats:moment3,proc_gyro:magnitude_stats:moment4,proc_gyro:magnitude_stats:percentile25,proc_gyro:magnitude_stats:percentile50,proc_gyro:magnitude_stats:percentile75,proc_gyro:magnitude_stats:value_entropy,proc_gyro:magnitude_stats:time_entropy,proc_gyro:magnitude_spectrum:log_energy_band0,proc_gyro:magnitude_spectrum:log_energy_band1,proc_gyro:magnitude_spectrum:log_energy_band2,proc_gyro:magnitude_spectrum:log_energy_band3,...,label:IN_A_CAR,label:ON_A_BUS,label:DRIVE_-_I_M_THE_DRIVER,label:DRIVE_-_I_M_A_PASSENGER,label:LOC_home,label:FIX_restaurant,label:PHONE_IN_POCKET,label:OR_exercise,label:COOKING,label:SHOPPING,label:STROLLING,label:DRINKING__ALCOHOL_,label:BATHING_-_SHOWER,label:CLEANING,label:DOING_LAUNDRY,label:WASHING_DISHES,label:WATCHING_TV,label:SURFING_THE_INTERNET,label:AT_A_PARTY,label:AT_A_BAR,label:LOC_beach,label:SINGING,label:TALKING,label:COMPUTER_WORK,label:EATING,label:TOILET,label:GROOMING,label:DRESSING,label:AT_THE_GYM,label:STAIRS_-_GOING_UP,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
Mean,1.439580e+09,0.996140,0.030828,0.027884,0.060992,0.981520,0.994155,1.007282,1.882186,6.682183,5.040875,0.055917,0.246215,0.151167,0.242377,0.452311,2.019172,0.181994,-0.046077,0.035470,-0.749838,0.043939,0.050663,0.046673,-0.059523,0.033502,0.002197,0.127447,0.143835,0.200107,0.283149,0.049316,0.083394,0.149725,1.868316,6.309744,5.182152,2.521560,3.336805,2.851731,...,0.086704,0.004806,0.084501,NaN,0.452013,0.007809,0.056626,0.019423,0.006408,NaN,NaN,NaN,0.003805,NaN,NaN,0.000801,0.005406,NaN,NaN,0.027433,0.001127,NaN,0.014417,0.158390,0.020224,NaN,NaN,0.002203,NaN,NaN,0.0002,NaN,0.239754,NaN,0.140986,0.023112,0.779276,NaN,NaN,1.911111
Variance,3.143862e+10,0.000358,0.004847,0.007567,0.017973,0.001276,0.000254,0.002323,0.419875,0.000107,0.000173,0.035224,0.482472,0.190299,0.364873,0.007409,6.591900,0.018673,0.060529,0.158416,0.177315,0.008062,0.012496,0.009386,0.093676,0.082347,0.105272,0.102261,0.090904,0.195666,0.380648,0.035204,0.077210,0.167317,0.412964,0.247884,0.182567,1.743231,0.999275,0.474556,...,0.079202,0.004784,0.077376,NaN,0.247737,0.007750,0.053440,0.019050,0.006368,NaN,NaN,NaN,0.003791,NaN,NaN,0.000800,0.005378,NaN,NaN,0.026686,0.001126,NaN,0.014212,0.133329,0.019819,NaN,NaN,0.002198,NaN,NaN,0.0002,NaN,0.182309,NaN,0.121156,0.022587,0.172071,NaN,NaN,3.871949


In [ ]:
df_test2 = load_data_for_user('/content/drive/MyDrive/Extrasensory/data/E65577C1-8D5D-4F70-AF23-B3ADB9D3DBA3.features_labels.csv')
X_test, y_test = get_features_and_target(df_test2, acc_sensors, target_column)
print(f'{y_train.shape[0]} examples with {y_train.sum()} positives')
X_test = imputer.transform(scaler.transform(X_test))
print(f'Test accuracy: {clf.score(X_test, y_test):0.4f}')
y_pred = clf.predict(X_test)
print(f'Balanced accuracy (train): {metrics.balanced_accuracy_score(y_test, y_pred):0.4f}')
pd.DataFrame([df_test2.mean(), df_test2.var()], index=['Mean', 'Variance'])

115048 examples with 10045.0 positives
Test accuracy: 0.9324
Balanced accuracy (train): 0.5622


,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,raw_acc:magnitude_spectrum:log_energy_band1,raw_acc:magnitude_spectrum:log_energy_band2,raw_acc:magnitude_spectrum:log_energy_band3,raw_acc:magnitude_spectrum:log_energy_band4,raw_acc:magnitude_spectrum:spectral_entropy,raw_acc:magnitude_autocorrelation:period,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,proc_gyro:magnitude_stats:mean,proc_gyro:magnitude_stats:std,proc_gyro:magnitude_stats:moment3,proc_gyro:magnitude_stats:moment4,proc_gyro:magnitude_stats:percentile25,proc_gyro:magnitude_stats:percentile50,proc_gyro:magnitude_stats:percentile75,proc_gyro:magnitude_stats:value_entropy,proc_gyro:magnitude_stats:time_entropy,proc_gyro:magnitude_spectrum:log_energy_band0,proc_gyro:magnitude_spectrum:log_energy_band1,proc_gyro:magnitude_spectrum:log_energy_band2,proc_gyro:magnitude_spectrum:log_energy_band3,...,label:IN_A_CAR,label:ON_A_BUS,label:DRIVE_-_I_M_THE_DRIVER,label:DRIVE_-_I_M_A_PASSENGER,label:LOC_home,label:FIX_restaurant,label:PHONE_IN_POCKET,label:OR_exercise,label:COOKING,label:SHOPPING,label:STROLLING,label:DRINKING__ALCOHOL_,label:BATHING_-_SHOWER,label:CLEANING,label:DOING_LAUNDRY,label:WASHING_DISHES,label:WATCHING_TV,label:SURFING_THE_INTERNET,label:AT_A_PARTY,label:AT_A_BAR,label:LOC_beach,label:SINGING,label:TALKING,label:COMPUTER_WORK,label:EATING,label:TOILET,label:GROOMING,label:DRESSING,label:AT_THE_GYM,label:STAIRS_-_GOING_UP,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
Mean,1.442470e+09,1.030703,0.071430,0.081863,0.129443,0.987937,1.019669,1.059327,2.127741,6.671697,5.033124,0.156208,0.335376,0.277838,0.468664,0.532067,2.750765,0.178879,-0.078755,0.011236,-0.834115,0.079610,0.070032,0.047252,-0.024843,-0.038040,-0.053231,0.117984,0.119116,0.166097,0.237703,0.049172,0.084074,0.146313,2.184907,6.430319,5.053236,1.957567,2.964670,2.671750,...,0.022211,NaN,NaN,NaN,0.222319,NaN,NaN,0.132280,0.004936,NaN,NaN,NaN,0.015301,0.005429,NaN,0.000494,0.009872,0.005429,NaN,NaN,NaN,NaN,0.044916,0.082428,0.061204,0.004936,0.019743,0.036525,NaN,NaN,0.001481,NaN,0.176209,NaN,NaN,1.0,NaN,0.000987,NaN,1.332752
Variance,4.416049e+09,0.004604,0.040021,0.059240,0.120609,0.006142,0.001298,0.023534,0.355261,0.002058,0.001701,0.239927,0.751834,0.583945,1.590369,0.119818,11.056375,0.020090,0.102721,0.053054,0.151621,0.044358,0.033604,0.011268,0.054759,0.053007,0.074843,0.093660,0.087578,0.185020,0.367706,0.025894,0.065863,0.161295,0.338861,0.206654,0.101773,1.348116,0.784848,0.332418,...,0.021729,NaN,NaN,NaN,0.172944,NaN,NaN,0.114839,0.004914,NaN,NaN,NaN,0.015074,0.005403,NaN,0.000494,0.009779,0.005403,NaN,NaN,NaN,NaN,0.042920,0.075671,0.057487,0.004914,0.019363,0.035208,NaN,NaN,0.001479,NaN,0.145231,NaN,NaN,0.0,NaN,0.000987,NaN,5.951162


In [ ]:
df_test3 = load_data_for_user('/content/drive/MyDrive/Extrasensory/data/ECECC2AB-D32F-4F90-B74C-E12A1C69BBE2.features_labels.csv')
X_test, y_test = get_features_and_target(df_test3, acc_sensors, target_column)
print(f'{y_train.shape[0]} examples with {y_train.sum()} positives')
X_test = imputer.transform(scaler.transform(X_test))
print(f'Test accuracy: {clf.score(X_test, y_test):0.4f}')
y_pred = clf.predict(X_test)
print(f'Balanced accuracy (train): {metrics.balanced_accuracy_score(y_test, y_pred):0.4f}')
pd.DataFrame([df_test3.mean(), df_test3.var()], index=['Mean', 'Variance'])

115048 examples with 10045.0 positives
Test accuracy: 0.9241
Balanced accuracy (train): 0.7508


,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,raw_acc:magnitude_spectrum:log_energy_band1,raw_acc:magnitude_spectrum:log_energy_band2,raw_acc:magnitude_spectrum:log_energy_band3,raw_acc:magnitude_spectrum:log_energy_band4,raw_acc:magnitude_spectrum:spectral_entropy,raw_acc:magnitude_autocorrelation:period,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,proc_gyro:magnitude_stats:mean,proc_gyro:magnitude_stats:std,proc_gyro:magnitude_stats:moment3,proc_gyro:magnitude_stats:moment4,proc_gyro:magnitude_stats:percentile25,proc_gyro:magnitude_stats:percentile50,proc_gyro:magnitude_stats:percentile75,proc_gyro:magnitude_stats:value_entropy,proc_gyro:magnitude_stats:time_entropy,proc_gyro:magnitude_spectrum:log_energy_band0,proc_gyro:magnitude_spectrum:log_energy_band1,proc_gyro:magnitude_spectrum:log_energy_band2,proc_gyro:magnitude_spectrum:log_energy_band3,...,label:IN_A_CAR,label:ON_A_BUS,label:DRIVE_-_I_M_THE_DRIVER,label:DRIVE_-_I_M_A_PASSENGER,label:LOC_home,label:FIX_restaurant,label:PHONE_IN_POCKET,label:OR_exercise,label:COOKING,label:SHOPPING,label:STROLLING,label:DRINKING__ALCOHOL_,label:BATHING_-_SHOWER,label:CLEANING,label:DOING_LAUNDRY,label:WASHING_DISHES,label:WATCHING_TV,label:SURFING_THE_INTERNET,label:AT_A_PARTY,label:AT_A_BAR,label:LOC_beach,label:SINGING,label:TALKING,label:COMPUTER_WORK,label:EATING,label:TOILET,label:GROOMING,label:DRESSING,label:AT_THE_GYM,label:STAIRS_-_GOING_UP,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
Mean,1.447656e+09,1.004368,0.043286,0.027884,0.071633,0.978324,1.003259,1.025035,1.940699,6.678999,5.038230,0.040090,0.353182,0.227025,0.266676,0.467091,2.440472,0.166221,0.052272,0.019755,-0.181001,0.052044,0.045250,0.053294,0.007855,-0.015318,-0.042996,0.146833,0.142474,0.186709,0.265283,0.060898,0.106172,0.182817,2.209745,6.404642,5.133317,2.055895,3.023080,2.620282,...,0.028271,0.013732,0.028271,NaN,0.218697,0.023425,0.008000,NaN,0.063813,0.012116,NaN,NaN,NaN,NaN,NaN,0.063813,NaN,NaN,NaN,NaN,NaN,0.028271,0.396607,NaN,0.100969,0.011309,NaN,NaN,NaN,NaN,NaN,NaN,0.081583,0.759289,NaN,0.279111,0.712889,0.080775,0.627625,0.059207
Variance,5.049712e+10,0.002031,0.010747,0.007167,0.020418,0.005622,0.002146,0.010108,0.420086,0.000468,0.000609,0.024184,0.991909,0.383342,0.337453,0.015881,7.544563,0.018886,0.173568,0.154972,0.617459,0.012387,0.008756,0.013059,0.087620,0.071762,0.082860,0.162646,0.106538,0.186534,0.374804,0.046776,0.132436,0.292203,0.370421,0.155191,0.138322,1.746115,1.048836,0.424723,...,0.027494,0.013554,0.027494,NaN,0.170917,0.022895,0.007943,NaN,0.059789,0.011979,NaN,NaN,NaN,NaN,NaN,0.059789,NaN,NaN,NaN,NaN,NaN,0.027494,0.239503,NaN,0.090848,0.011190,NaN,NaN,NaN,NaN,NaN,NaN,0.074988,0.182917,NaN,0.201387,0.204860,0.074311,0.233901,2.101622


In [ ]:
df_test4 = load_data_for_user('/content/drive/MyDrive/Extrasensory/data/F50235E0-DD67-4F2A-B00B-1F31ADA998B9.features_labels.csv')
X_test, y_test = get_features_and_target(df_test4, acc_sensors, target_column)
print(f'{y_train.shape[0]} examples with {y_train.sum()} positives')
X_test = imputer.transform(scaler.transform(X_test))
print(f'Test accuracy: {clf.score(X_test, y_test):0.4f}')
y_pred = clf.predict(X_test)
print(f'Balanced accuracy (train): {metrics.balanced_accuracy_score(y_test, y_pred):0.4f}')
pd.DataFrame([df_test4.mean(), df_test4.var()], index=['Mean', 'Variance'])

115048 examples with 10045.0 positives
Test accuracy: 0.9195
Balanced accuracy (train): 0.5675


,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,raw_acc:magnitude_spectrum:log_energy_band1,raw_acc:magnitude_spectrum:log_energy_band2,raw_acc:magnitude_spectrum:log_energy_band3,raw_acc:magnitude_spectrum:log_energy_band4,raw_acc:magnitude_spectrum:spectral_entropy,raw_acc:magnitude_autocorrelation:period,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,proc_gyro:magnitude_stats:mean,proc_gyro:magnitude_stats:std,proc_gyro:magnitude_stats:moment3,proc_gyro:magnitude_stats:moment4,proc_gyro:magnitude_stats:percentile25,proc_gyro:magnitude_stats:percentile50,proc_gyro:magnitude_stats:percentile75,proc_gyro:magnitude_stats:value_entropy,proc_gyro:magnitude_stats:time_entropy,proc_gyro:magnitude_spectrum:log_energy_band0,proc_gyro:magnitude_spectrum:log_energy_band1,proc_gyro:magnitude_spectrum:log_energy_band2,proc_gyro:magnitude_spectrum:log_energy_band3,...,label:IN_A_CAR,label:ON_A_BUS,label:DRIVE_-_I_M_THE_DRIVER,label:DRIVE_-_I_M_A_PASSENGER,label:LOC_home,label:FIX_restaurant,label:PHONE_IN_POCKET,label:OR_exercise,label:COOKING,label:SHOPPING,label:STROLLING,label:DRINKING__ALCOHOL_,label:BATHING_-_SHOWER,label:CLEANING,label:DOING_LAUNDRY,label:WASHING_DISHES,label:WATCHING_TV,label:SURFING_THE_INTERNET,label:AT_A_PARTY,label:AT_A_BAR,label:LOC_beach,label:SINGING,label:TALKING,label:COMPUTER_WORK,label:EATING,label:TOILET,label:GROOMING,label:DRESSING,label:AT_THE_GYM,label:STAIRS_-_GOING_UP,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
Mean,1.442131e+09,1.061734,0.037133,0.026543,0.066620,1.041599,1.057482,1.079899,1.923716,6.682415,5.041476,0.064635,0.211311,0.124328,0.226923,0.448750,1.845320,0.152422,0.034615,0.094799,0.871232,0.050038,0.047431,0.043280,-0.020796,-0.016036,-0.067708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.013483,NaN,0.030562,0.209620,NaN,NaN,0.026517,0.014831,0.012584,0.000899,NaN,0.013933,0.014382,0.002247,0.012584,0.058876,0.021573,0.000899,NaN,NaN,NaN,0.001348,0.190562,0.067865,0.014831,0.004494,NaN,NaN,NaN,NaN,NaN,0.204229,0.008989,NaN,NaN,NaN,NaN,NaN,2.957193
Variance,3.171886e+10,0.010254,0.010217,0.011741,0.024493,0.002034,0.005278,0.027318,0.265722,0.000112,0.000184,0.059506,0.428547,0.172758,0.281415,0.005353,3.145275,0.015514,0.121156,0.080453,0.124911,0.023325,0.015282,0.008301,0.060821,0.061367,0.075870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.013307,NaN,0.029641,0.165753,NaN,NaN,0.025825,0.014618,0.012431,0.000898,NaN,0.013745,0.014182,0.002243,0.012431,0.055435,0.021117,0.000898,NaN,NaN,NaN,0.001347,0.154317,0.063288,0.014618,0.004476,NaN,NaN,NaN,NaN,NaN,0.162592,0.008912,NaN,NaN,NaN,NaN,NaN,2.441875


In [ ]:
df_test5 = load_data_for_user('/content/drive/MyDrive/Extrasensory/data/FDAA70A1-42A3-4E3F-9AE3-3FDA412E03BF.features_labels.csv')
X_test, y_test = get_features_and_target(df_test5, acc_sensors, target_column)
print(f'{y_train.shape[0]} examples with {y_train.sum()} positives')
X_test = imputer.transform(scaler.transform(X_test))
print(f'Test accuracy: {clf.score(X_test, y_test):0.4f}')
y_pred = clf.predict(X_test)
print(f'Balanced accuracy (train): {metrics.balanced_accuracy_score(y_test, y_pred):0.4f}')
pd.DataFrame([df_test5.mean(), df_test5.var()], index=['Mean', 'Variance'])

115048 examples with 10045.0 positives
Test accuracy: 0.9082
Balanced accuracy (train): 0.5580


,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,raw_acc:magnitude_spectrum:log_energy_band1,raw_acc:magnitude_spectrum:log_energy_band2,raw_acc:magnitude_spectrum:log_energy_band3,raw_acc:magnitude_spectrum:log_energy_band4,raw_acc:magnitude_spectrum:spectral_entropy,raw_acc:magnitude_autocorrelation:period,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,proc_gyro:magnitude_stats:mean,proc_gyro:magnitude_stats:std,proc_gyro:magnitude_stats:moment3,proc_gyro:magnitude_stats:moment4,proc_gyro:magnitude_stats:percentile25,proc_gyro:magnitude_stats:percentile50,proc_gyro:magnitude_stats:percentile75,proc_gyro:magnitude_stats:value_entropy,proc_gyro:magnitude_stats:time_entropy,proc_gyro:magnitude_spectrum:log_energy_band0,proc_gyro:magnitude_spectrum:log_energy_band1,proc_gyro:magnitude_spectrum:log_energy_band2,proc_gyro:magnitude_spectrum:log_energy_band3,...,label:IN_A_CAR,label:ON_A_BUS,label:DRIVE_-_I_M_THE_DRIVER,label:DRIVE_-_I_M_A_PASSENGER,label:LOC_home,label:FIX_restaurant,label:PHONE_IN_POCKET,label:OR_exercise,label:COOKING,label:SHOPPING,label:STROLLING,label:DRINKING__ALCOHOL_,label:BATHING_-_SHOWER,label:CLEANING,label:DOING_LAUNDRY,label:WASHING_DISHES,label:WATCHING_TV,label:SURFING_THE_INTERNET,label:AT_A_PARTY,label:AT_A_BAR,label:LOC_beach,label:SINGING,label:TALKING,label:COMPUTER_WORK,label:EATING,label:TOILET,label:GROOMING,label:DRESSING,label:AT_THE_GYM,label:STAIRS_-_GOING_UP,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
Mean,1.443986e+09,1.014874,0.081997,0.103286,0.172768,0.983236,1.004754,1.034813,1.685263,6.675351,5.034582,0.143275,0.591190,0.437614,0.652608,0.514475,2.440548,0.168984,0.008016,0.293367,-0.273169,0.083060,0.105511,0.103546,-0.012141,0.023255,0.006260,0.324565,0.310746,0.387163,0.536730,0.122148,0.225487,0.424419,1.935883,6.212036,5.426419,2.941502,3.555685,2.874598,...,NaN,NaN,0.066306,NaN,NaN,NaN,0.462349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006766,NaN,NaN,NaN,NaN,NaN,0.009472,0.105548,0.057285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.149676,0.419035,0.019484,NaN,0.518167,NaN,NaN,0.343455
Variance,4.330783e+10,0.003035,0.021721,0.036954,0.077626,0.001165,0.001301,0.013057,0.500192,0.000735,0.001017,0.099699,1.132268,0.602226,1.171012,0.050185,7.624135,0.010229,0.071947,0.217044,0.517802,0.012831,0.030218,0.022274,0.101095,0.107606,0.139947,0.370292,0.180137,0.253209,0.496591,0.117925,0.284510,0.679570,0.424045,0.194823,0.239574,1.816375,0.806580,0.410532,...,NaN,NaN,0.061937,NaN,NaN,NaN,0.248713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006723,NaN,NaN,NaN,NaN,NaN,0.009387,0.094450,0.054027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.127332,0.243554,0.019114,NaN,0.249801,NaN,NaN,2.261742


Combining 5 different Users to reduce Variance

In [ ]:
a1= df_test1.append(df_test2)
a2= a1.append(df_test3)
a3= a2.append(df_test4)
a4= a3.append(df_test5)
pd.DataFrame([a4.mean(), a4.var()], index= ['Mean', 'variance'])

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,raw_acc:magnitude_spectrum:log_energy_band1,raw_acc:magnitude_spectrum:log_energy_band2,raw_acc:magnitude_spectrum:log_energy_band3,raw_acc:magnitude_spectrum:log_energy_band4,raw_acc:magnitude_spectrum:spectral_entropy,raw_acc:magnitude_autocorrelation:period,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,proc_gyro:magnitude_stats:mean,proc_gyro:magnitude_stats:std,proc_gyro:magnitude_stats:moment3,proc_gyro:magnitude_stats:moment4,proc_gyro:magnitude_stats:percentile25,proc_gyro:magnitude_stats:percentile50,proc_gyro:magnitude_stats:percentile75,proc_gyro:magnitude_stats:value_entropy,proc_gyro:magnitude_stats:time_entropy,proc_gyro:magnitude_spectrum:log_energy_band0,proc_gyro:magnitude_spectrum:log_energy_band1,proc_gyro:magnitude_spectrum:log_energy_band2,proc_gyro:magnitude_spectrum:log_energy_band3,...,label:IN_A_CAR,label:ON_A_BUS,label:DRIVE_-_I_M_THE_DRIVER,label:DRIVE_-_I_M_A_PASSENGER,label:LOC_home,label:FIX_restaurant,label:PHONE_IN_POCKET,label:OR_exercise,label:COOKING,label:SHOPPING,label:STROLLING,label:DRINKING__ALCOHOL_,label:BATHING_-_SHOWER,label:CLEANING,label:DOING_LAUNDRY,label:WASHING_DISHES,label:WATCHING_TV,label:SURFING_THE_INTERNET,label:AT_A_PARTY,label:AT_A_BAR,label:LOC_beach,label:SINGING,label:TALKING,label:COMPUTER_WORK,label:EATING,label:TOILET,label:GROOMING,label:DRESSING,label:AT_THE_GYM,label:STAIRS_-_GOING_UP,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
Mean,1.442819e+09,1.015212,0.052990,0.055204,0.102225,0.989116,1.009620,1.033869,1.890319,6.678226,5.037645,0.092333,0.359922,0.252452,0.382954,0.483062,2.298767,0.172299,-0.012470,0.098062,-0.370155,0.061559,0.065996,0.061395,-0.026197,0.005021,-0.021708,0.183420,0.184607,0.242296,0.340525,0.071492,0.126876,0.230761,2.013223,6.324286,5.215134,2.439147,3.265224,2.778876,...,0.062122,0.008395,0.071488,0.030562,0.311970,0.010911,0.183986,0.045863,0.014690,0.012417,0.000899,NaN,0.008761,0.010115,0.002247,0.010684,0.016838,0.013879,0.000899,0.027433,0.001127,0.028271,0.053386,0.140988,0.049449,0.010384,0.011762,0.012108,NaN,NaN,0.00057,NaN,0.192016,0.332570,0.089655,0.110314,0.677758,0.031250,0.627625,1.227816
variance,7.671798e+12,0.003520,0.016954,0.024955,0.053097,0.003255,0.001924,0.013195,0.431780,0.000668,0.000723,0.088478,0.787680,0.401886,0.783719,0.039116,7.467216,0.016627,0.098816,0.158507,0.585358,0.018067,0.020596,0.013936,0.084815,0.080149,0.102287,0.193367,0.123824,0.215661,0.423530,0.059328,0.146303,0.345049,0.415537,0.214388,0.193111,1.832644,0.968723,0.431711,...,0.058270,0.008326,0.066385,0.029641,0.214659,0.010794,0.150162,0.043764,0.014476,0.012266,0.000898,NaN,0.008686,0.010015,0.002243,0.010571,0.016556,0.013690,0.000898,0.026686,0.001126,0.027494,0.050540,0.121121,0.047007,0.010278,0.011626,0.011963,NaN,NaN,0.00057,NaN,0.155158,0.222006,0.081635,0.098171,0.218441,0.030283,0.233901,4.267114
